# การ train แบบ Teacher Forcing

In [1]:
# นำเข้า library ที่จำเป็น
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import pickle


In [2]:
# กำหนด device ของ PyTorch เป็น GPU
device = torch.device("cuda")

## ***Datasets***

Note : window size is 14 according to the length of shortest note sequence

In [3]:
# ทำการอ่าน DataFrame ที่ได้ preprocess เอาไว้สำหรับการเทรนแบบ teacher forcing
df = pd.read_pickle("../data/dataset_undertale_preprocessed.pkl")
# ลองแสดงผล 3 แถวแรก
df.head(3)

,Path,Data
0,../data/96_res\006_Uwa!!_So_Temperate.mid,"[[G#3, G6, G#3, G6, G#3, G6, C4, F6, C4, F6, C..."
1,../data/96_res\007_Anticipation.mid,"[[6.7.9.11, G2, 6.7.11, G2, 6.7.11, G2, 6.7.11..."
2,../data/96_res\008_Unnecessary_Tension.mid,"[[1.2, 1.6, 1.2.6, 1, 1.6, 1.7, 1, 1, 6.9.1, 1..."


In [4]:
# รูปแบบชองชุดข้อมูล custom สำหรับ teacher forcing
class UndertaleDataset(Dataset): # ทำการสร้าง class UndertaleDataset จาก Dataset ของ PyTorch

    def __init__(self, df, unique_notes_path, use_embedding=False, subset=None): # กำหนด argument ที่จำเป็น

        super(UndertaleDataset, self).__init__() # super class constructor

        self.df = df # DataFrame
        self.path = list(df["Path"]) # ตำแหน่งที่เก็บไฟล์เพลง
        self.raw_data = [np.array(music) for music in df["Data"]] # ข้อมูลดิบของ sequence ทั้งหมด -> แปลงเป็น numpy -> เก็บใน list

        self.use_embedding = use_embedding # กำหนดการแปลง label ว่าให้สอบคล้องกับ Embedding layer หรือไม่

        if subset:

            self.raw_data = self.raw_data[:subset] # Subset ใช้แบ่ง data ให้เล็กลง เพื่อทดลอง train ก่อนการ train จริง

        with open(unique_notes_path, "rb") as f:

            self.unique_notes = pickle.load(f) # โหลด ไฟล์ที่เก็บโน๊ตที่เป็นไปได้
            

        self.pitchname = sorted(self.unique_notes)
        self.pitchname.insert(0,"<S>") # ทำการเพิ่ม start token สำหรับการ generate *** เป็นเอกลักษณ์ของ teacher forcing ***
        # "<S>" จะถูกกำหนดให้เป็น token ลำดับแรก หรือ index=0
        self.note_to_int = dict((note, number) for number, note in enumerate(self.pitchname))# note -> int
        self.int_to_note = dict((number, note) for number, note in enumerate(self.pitchname))# int -> note
        self.n = len(self.pitchname)
        self.all_data = np.vectorize(lambda x: self.note_to_int[x])(np.concatenate(self.raw_data)) # map note_to_int ใน dataset

        self.label = torch.tensor(self.all_data).long() # แปลงเป็น tensor ของจำนวนเต็ม

        self.input = torch.tensor(np.append(np.zeros((self.all_data.shape[0], 1)), self.all_data[:, :-1], axis=1)).long() # เติม start token ให้กับ input ของโมเดล และนำโน๊ตตัวสุดท้ายของ seq. ออก

    def __len__(self):
        return len(self.all_data) # ทำให้สามารถเรียกใช้ len(dataset) ได้
 
    def __getitem__(self, index):

        x, y = self.input[index], self.label[index]

        if self.use_embedding:
            
            return x, y # หากมีการใช้ embedding layer จะรับ input เป็น int หรือ long tensor

        else:

            x = F.one_hot(x % self.n, num_classes=self.n).float()

            return x, y # หากไม่มีการใช้ embedding layer จะรับ input เป็น onehot
        



In [5]:
# กำหนด dataset
ds = UndertaleDataset(df=df, unique_notes_path="../data/all_undertale_unique_notes.pkl", use_embedding=True)

In [6]:
# save dictionary ที่แปลงจาก int เป็น note ไว้สำหรับการ inference

with open("../data/teacher_forcing_int_to_note.pkl", "wb") as f:
    pickle.dump(ds.int_to_note, f)

In [ ]:
### demo of input size
x, y = ds[0]
x, y

In [7]:
# กำหนด dataloader
train_loader = DataLoader(ds, batch_size=128, shuffle=True)

## ***Model***

In [8]:
class BasicMusicLSTM(nn.Module):

    def __init__(self , Tx, embedding_size, n_hidden, vocab_size, test_size=5):

        super(BasicMusicLSTM, self).__init__()

        self.Tx = Tx
        self.embedding_size = embedding_size
        self.n_hidden = n_hidden
        self.vocab_size = vocab_size
        self.test_size = test_size

        self.embedding = torch.nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embedding_size, padding_idx=0) # Embedding layer ที่กำหนดให้ผลลัพธ์ของ <S> เป็น vector 0
        # LSTM cell 2 ชั้น
        self.lstm_1 = torch.nn.LSTMCell(input_size=self.embedding_size, hidden_size=n_hidden) 
        self.lstm_2 = torch.nn.LSTMCell(input_size=n_hidden, hidden_size=n_hidden)
        # classifier ที่มีขนาดเท่ากับจำนวนโน๊ตที่เป็นไปได้ทั้งหมด
        self.dense = torch.nn.Linear(n_hidden, vocab_size)

    # กำหนดค่า hidden state และ cell state เริ่มต้นของ lstm -> เบื้องต้นเป็น vector 0
    def _init_hidden(self, batch, size):
        
        return torch.zeros(batch, size).to(device), torch.zeros(batch, size).to(device)
    # กำหนดค่า start token สำหรับการ sampling generate ระหว่างการ train
    def _init_x0(self):

        x0 = torch.zeros(self.test_size).to(device)
    

        return x0

    def forward(self, x):

        if self.training: # ใน mode training ของ model
            
            self.batch = x.shape[0] # กำหนด batch
            # สร้าง hidden state และ cell state
            h0, c0 = self._init_hidden(self.batch, self.n_hidden)
            h1, c1 = self._init_hidden(self.batch, self.n_hidden)

            #นำ input ผ่าน embedding layer
            x = self.embedding(x)

            output = []

            for t in range(self.Tx): # loop ผ่าน sequence    

                x_t = x[:, t]

                x_t = x_t.view((self.batch, self.embedding_size))

                h0, c0 = self.lstm_1(x_t, (h0, c0))

                h1, c1 = self.lstm_2(h0, (h1, c1))

                out = self.dense(h1)

                #out = self.dense(h0)

                out = torch.unsqueeze(out, dim=0)

                output.append(out)


            return output
        
        else: # ใน model evaluation ของโมเดล

            x = self._init_x0().to(device).long() # กำหนด x0 (start token)
            # สร้าง hidden state และ cell state
            h0, c0 = self._init_hidden(self.test_size, self.n_hidden)
            h1, c1 = self._init_hidden(self.test_size, self.n_hidden)

            #นำ input ผ่าน embedding layer
            x = self.embedding(x)
            output = []

            for t in range(self.Tx): # loop ผ่าน sequence    


                ###x_t = x[:, t, :]

                ###x_t = x_t.view((self.batch, self.vocab_size))

                h0, c0 = self.lstm_1(x, (h0, c0))

                h1, c1 = self.lstm_2(h0, (h1, c1))

                out = self.dense(h1)

                ###out = self.dense(h0)

                out = torch.unsqueeze(out, dim=0)

                output.append(out)

                x = self.embedding(torch.multinomial(F.softmax(torch.squeeze(out), dim=-1), 1).long()) # ทำการสุ่มแบบ multinomial เพื่อทำนายโน๊ตตัวถัดไป

                x = x.view((self.test_size, self.embedding_size)) # นำโน๊ตตัวที่ได้จากการทำนายเป็น input ของ step ถัดไป


            return output




In [9]:
# function แสดงผล note ที่ generate ระหว่างการ train
def generate_seq(model_inferencing_output):
    seq_list = []
    for seq in model_inferencing_output:

        indices = torch.argmax(seq, dim=-1).tolist()

        text_seq = " | ".join([ds.int_to_note[r] for  r in indices])

        seq_list.append(text_seq)

    return seq_list

In [ ]:
# กำหนดโมเดล
model = BasicMusicLSTM(Tx=14, embedding_size=50, n_hidden=256, vocab_size=ds.n, test_size=5).to(device)

In [ ]:
model

## ***Training***

In [ ]:
# กำหนด hyperparameters สำหรับการ train
hyperparam = {
    "num_epochs": 200,
    "lr" : 1e-3,
    "optimizer" : torch.optim.Adam,
}

In [ ]:
# กำหนด loss function และ optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = hyperparam["optimizer"](model.parameters(), lr=hyperparam["lr"])

In [ ]:
## Training loop
print(f"Training with hyperparameters : \n{hyperparam}")
running_training_loss = []
print("Start Training!")
for i in range(hyperparam["num_epochs"]):

    model.train() # ตั้ง model เป็น mode training
    pbar = tqdm(train_loader)
    pbar.set_description(f"Epoch - {i + 1} / {hyperparam['num_epochs']}")
    
    epoch_loss = 0
    for data, label in pbar:   
        
        data, label = data.to(device), label.to(device)

        optimizer.zero_grad()

        pred = torch.cat(model(data), axis=0).permute((1, 2, 0))


        loss = loss_fn(pred, label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item() * data.size(0)
        pbar.set_postfix({"loss" : loss.item()})

    print(f"Average Training CrossEntropyLoss of Epoch {i + 1} : {epoch_loss / len(train_loader.dataset)}")
    running_training_loss.append(epoch_loss / len(train_loader.dataset))


    if (i+1) % 10 == 0:
        model.eval() # ตั้ง model เป็น mode evaluation
        print()
        print(f"---------------- Generating text after Epoch : {i + 1}")
        generate = torch.swapaxes(torch.cat(model(data), axis=0), 0, 1)
        seq_list = generate_seq(generate)

        for i, seq in enumerate(seq_list):
            print(f"{i + 1}) {seq}")



In [ ]:
torch.save(model.state_dict(), "teacher_forcing_SimpleLSTM_256_emb50_padding_emb_undertale_200_epochs.pt")

# **Inference**

In [14]:
inference_model = BasicMusicLSTM(Tx=200, embedding_size=50, n_hidden=256, vocab_size=ds.n, test_size=50).to(device)

In [15]:
inference_model.load_state_dict(torch.load("teacher_forcing_SimpleLSTM_256_emb50_padding_emb_undertale_200_epochs.pt"))

<All keys matched successfully>

In [16]:
inference_model.eval()

music_array = torch.transpose(torch.cat(inference_model(torch.randn((1)).to(device)), dim=0), 1, 0)

In [17]:
music_array.shape

torch.Size([50, 200, 1141])

In [18]:
### utils function that writes .midi files

def create_midi(prediction_output, filename):
    from music21 import note, chord, instrument, stream
    offset = 0
    output_notes = []

        # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
            # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

            # increase offset each iteration so that notes do not stack
        offset += 0.25

    midi_stream = stream.Stream(output_notes)

    midi_stream.write("midi", fp=filename)
    return output_notes

In [19]:
midi_stream_list = []
for no, music in enumerate(music_array):
    prediction_output = torch.argmax(music, dim=-1)
    
    prediction_output = np.vectorize(lambda x: ds.int_to_note[x])(prediction_output.cpu())
    print(prediction_output)
    output_note = create_midi(prediction_output, f"eval_teacher_forcing_SimpleLSTM_256_emb50_undertale_{no + 1}.mid")

    midi_stream_list.append(output_note)


midi_stream_list

['F#2' 'F5' 'F2' 'F4' 'F2' 'B-2' 'F4' 'B-2' 'G#2' 'G#3' 'E-4' 'B-2' 'G#2'
 'G#3' 'E-4' 'D4' 'G#2' 'E-4' 'A2' 'A2' 'A2' '3.9' '9.1' '3.4' '3.4' '4'
 '3.7' '3.7' 'C2' 'A2' 'A2' 'A2' '9.1' '9.11' '9.11' '11.2' '9.11' '4.9'
 '11.4' '4.7.9' 'C#6' 'G#2' 'E-5' 'E-5' 'C4' 'D5' 'B-3' 'F#4' 'F#4' 'F#4'
 '6.9.11.2' '10.1' '6.10' '6.10' '8.11' '8.11' '8.11' 'E-3' '8.11' 'G#2'
 '8.11' 'G#2' '8.11' 'G#1' 'G#2' '8' 'F#2' '8' 'G#2' '8' '11.3' '11.3'
 '11.3' '11.3' 'G#4' '11.3' 'B-4' '11.3' 'F#4' '11.3' '11.3' 'E-5' 'F#2'
 'A1' '10.0' '8.11' '3.4' '3.4' '6.10' 'F#2' 'B-2' 'F#2' '10.1' 'B-2'
 '10.1' '10.1' '10.1' '10.1' '10.1' '11.3' 'F#4' '11.3' 'E4' '11.3' '8.11'
 'F#2' '11.3' '8.11' '3.6' 'B-2' '8' '3.6' '3.6' '8' '3.6' 'F#2' '3.6'
 'F#2' '8' 'C#3' '3.8' 'C#3' '3.8' 'C#3' 'B-3' 'C#3' 'B-3' 'C#3' 'B-3'
 'B-5' 'B-3' 'C#3' 'B-3' 'G#5' 'B-3' 'G#5' 'G#5' 'B-3' '5.10' 'B-3' 'G#5'
 'F#5' 'F#5' 'C#3' 'F#5' 'C#3' 'F5' 'G#5' 'F5' 'F5' 'F5' 'F5' 'B-3' 'F5'
 'B-3' 'F5' 'B-3' 'B-5' 'B-5' 'B-3' 'B-5' '7' 'G#4' 'C6

KeyboardInterrupt: 